In [1]:
import os
import cv2
import face_recognition
import numpy as np
import tkinter as tk
from tkinter import scrolledtext
from PIL import Image, ImageTk
from datetime import datetime
import locale

# Đặt lại môi trường locale để hỗ trợ Unicode
locale.setlocale(locale.LC_ALL, 'vi_VN.UTF-8')

'vi_VN.UTF-8'

In [2]:
path = 'student_images'  # Thay đổi thành đường dẫn thư mục chứa hình ảnh 

In [4]:
images = []
classNames = []
classInfo = {}  # Từ điển để lưu thông tin từ tệp .txt
classNames1 = []
# Duyệt qua các thư mục và tệp tin trong `path`
for root, dirs, files in os.walk(path):
    for cl in dirs:  # Duyệt qua các thư mục con
        class_path = os.path.join(root, cl)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            if img_name.endswith(('png', 'jpg', 'jpeg')):
                curImg = cv2.imread(img_path)
                if curImg is not None:
                    images.append(curImg)
                    txt_path = os.path.join(class_path, cl + '.txt')
                    if os.path.exists(txt_path):
                        with open(txt_path, 'r', encoding='utf-8') as file:
                            content = file.read().strip().splitlines()
                            name = content[0].strip()  # Lấy tên từ dòng đầu tiên
                            info = '\n'.join(content[1:]).strip()  # Lấy thông tin từ các dòng còn lại
                            classNames.append(cl)
                            classNames1.append(name)
                            classInfo[name] = info
                    else:
                        print(f"Error: Could not find the text file {txt_path}")
                else:
                    print(f"Error: Could not load image {img_path}")

# Hiển thị số lượng hình ảnh đã được nạp
print(f"Loaded {len(images)} images and corresponding names and info.")

# Mã hóa tất cả các khuôn mặt trong images và lưu vào danh sách
encoded_face_train = [face_recognition.face_encodings(img)[0] for img in images]

# Đọc hình ảnh từ file để nhận diện
test_image_path = 'Giang.jpg'
img = cv2.imread(test_image_path)

# Thay đổi kích thước hình ảnh xuống còn 1/4 kích thước để xử lý nhanh hơn
imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)

# Chuyển đổi hình ảnh từ BGR (định dạng OpenCV) sang RGB (định dạng face_recognition)
imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

# Tìm tất cả các vị trí khuôn mặt trong hình ảnh
faces_in_frame = face_recognition.face_locations(imgS)

# Mã hóa các khuôn mặt được tìm thấy trong hình ảnh
encoded_faces = face_recognition.face_encodings(imgS, faces_in_frame)

# Tạo cửa sổ tkinter
root = tk.Tk()
root.title("Thông tin Sinh Viên")
root.geometry("1920x1080")

# Tạo vùng văn bản có thể cuộn
txt = scrolledtext.ScrolledText(root, width=40, height=20, font=('Arial', 12))  # Sử dụng font hỗ trợ Unicode
txt.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

# Tạo một nhãn để hiển thị hình ảnh
lbl = tk.Label(root)
lbl.pack(side=tk.LEFT)

# Biến lưu trữ số lượng khuôn mặt nhận diện
num_detected_faces = 0

# Lặp qua từng khuôn mặt được mã hóa và vị trí của nó
for encode_face, faceloc in zip(encoded_faces, faces_in_frame):
    # So sánh khuôn mặt được mã hóa với các khuôn mặt đã biết
    matches = face_recognition.compare_faces(encoded_face_train, encode_face)
    
    # Tính khoảng cách giữa khuôn mặt được mã hóa và các khuôn mặt đã biết
    faceDist = face_recognition.face_distance(encoded_face_train, encode_face)
    
    # Tìm chỉ số của khoảng cách nhỏ nhất (khuôn mặt giống nhất)
    matchIndex = np.argmin(faceDist)
    
    # Nếu tìm thấy sự phù hợp, tiến hành chú thích
    if matches[matchIndex]:
        name = classNames[matchIndex].upper()
        name1 = classNames1[matchIndex].upper()
        
        # Trích xuất vị trí khuôn mặt và mở rộng lại kích thước ban đầu
        y1, x2, y2, x1 = faceloc
        y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
        
        # Vẽ một hình chữ nhật xung quanh khuôn mặt
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Vẽ một hình chữ nhật đầy màu dưới khuôn mặt để hiển thị tên
        cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
        
        # Đặt tên của người đó dưới khuôn mặt
        cv2.putText(img, name, (x1 + 6, y2 - 5), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
        
        # Lấy thời gian hiện tại
        current_time = datetime.now().strftime(f"- Ngày %d-%m-%Y \n - Giờ %H:%M:%S")
        # Hiển thị thông tin từ tệp .txt trong giao diện tkinter

        info = classInfo.get(classNames1[matchIndex])
        # Hiển thị thông tin từ tệp .txt và thời gian nhận diện trong giao diện tkinter
        txt.insert(tk.END, f" Tên: {name1} \n MSSV: {info} \n Nhận dạng lúc: \n {current_time}\n\n")
        
        # Tăng biến đếm số lượng khuôn mặt nhận diện
        num_detected_faces += 1

# Kiểm tra nếu không có khuôn mặt nào được nhận diện
if num_detected_faces == 0:
    txt.insert(tk.END, "Không nhận diện được khuôn mặt nào trong ảnh.\n")

# Chuyển đổi hình ảnh từ BGR sang RGB để sử dụng với PIL
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_pil = Image.fromarray(img)
img_tk = ImageTk.PhotoImage(image=img_pil)

# Đặt hình ảnh vào nhãn
lbl.config(image=img_tk)
lbl.image = img_tk

# Khởi động giao diện tkinter
root.mainloop()


Loaded 5 images and corresponding names and info.


REAL-TIME WEBCCAM RECOGNIZATION

In [ ]:
# take pictures from webcam 
cap  = cv2.VideoCapture(0) 
while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0,0), None, 0.25,0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    faces_in_frame = face_recognition.face_locations(imgS)
    encoded_faces = face_recognition.face_encodings(imgS, faces_in_frame)
    for encode_face, faceloc in zip(encoded_faces,faces_in_frame):
        matches = face_recognition.compare_faces(encoded_face_train, encode_face)
        faceDist = face_recognition.face_distance(encoded_face_train, encode_face)
        matchIndex = np.argmin(faceDist)
        print(matchIndex)
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            y1,x2,y2,x1 = faceloc
            # since we scaled down by 4 times
            y1, x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img, (x1,y2-35),(x2,y2), (0,255,0), cv2.FILLED)
            cv2.putText(img,name, (x1+6,y2-5), cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
    cv2.imshow('webcam', img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break